In [1]:
import csv
from urllib.request import urlopen
from bs4 import BeautifulSoup
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import selenium
import pandas as pd 
import time
import os

In [2]:
def select_element_fx(name, value):
    element_name = driver.find_element_by_name(name)
    select_element = Select(element_name)
    select_element.select_by_value(value)

In [3]:
driver = webdriver.Chrome("chromedriver.exe")
driver.get('https://db.netkeiba.com/?pid=horse_search_detail')

# Define under age
select_element_fx('under_age', 'none')
# Define over age
select_element_fx('over_age', 'none')
# Define Displayed results
select_element_fx('list', '100')
# Click Genekiba
driver.find_element_by_id("check_other_01").click()

# Click Search Button
elements = driver.find_element_by_class_name("button")
driver.execute_script("arguments[0].click();", elements)

# get all URLs

time.sleep(0.1)
k = 0
l = 0
while True:
    url_list = []
    try:
        page = driver.find_elements_by_xpath('/html/body/div[1]/div[2]/div/div/div[2]')[0].text
        page =  page.split('件中')[1].split('件目')[0]
        pagemax = int(page.split('～')[1])-int(page.split('～')[0]) + 1
        for i in range(2, 2+ pagemax ,1):
            elem = driver.find_elements_by_xpath('//*[@id="contents_liquid"]/div/form/table/tbody/tr[{}]/td[2]/a'.format(i))[0].get_attribute("href")
            url_list.append(elem)

        if k == 0:
            driver.find_elements_by_xpath('/html/body/div[1]/div[2]/div/div/div[2]/a')[0].click()
            k += 1

        else:
            driver.find_elements_by_xpath('/html/body/div[1]/div[2]/div/div/div[2]/a[2]')[0].click()
    except IndexError:
        break

    pd.DataFrame(url_list, columns=['URL']).to_csv('\Uma{:07d}.csv'.format(l), encoding = "shift_jis")
    l += 1

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=87.0.4280.141)


In [32]:
pd.DataFrame(url_list, columns=['URL']).to_csv(r'\Uma{:07d}.csv'.format(l), encoding = "shift_jis")

# URLごとにダウンロードする

In [74]:
def uma_get(driver, url):

    driver.get(url)

    # Get Table information
    tableElem = driver.find_element_by_class_name("db_h_race_results")
    trs = tableElem.find_elements(By.TAG_NAME, "tr")

    # Get without Header
    res1_ = []
    for i in range(1,len(trs)):
        tds = trs[i].find_elements(By.TAG_NAME, "td")
        temp=[]
        for j in range(0,len(tds)):
            temp.append(tds[j].text)
        res1_.append(temp)

    col1 = ["日付", "開催", "天気", "R", "レース名", "映像", "頭数", "枠番", 
            "馬番", "オッズ", "人気", "着番", "騎手", "斤量", "距離", "馬場",
            "馬場指数", "タイム", "着差", "タイム指数", "通過", "ペース", 
           "上り", "馬体重", "厩舎コメント", "備考", "勝ち馬(2着馬)", "賞金"]

    df = pd.DataFrame(res1_, columns=col1)

    # Get Table information
    tableElem = driver.find_element_by_class_name("db_prof_table")
    trs = tableElem.find_elements(By.TAG_NAME, "tr")

    # Get without Header
    res2_ = []
    for i in range(0,len(trs)):
        tds = trs[i].find_elements(By.TAG_NAME, "td")
        for j in range(0,len(tds)):
            res2_.append(tds[j].text)
            
    uma_name = driver.find_elements_by_xpath("/html/body/div[1]/div[2]/div[2]/div[1]/div[1]/div[1]/h1")[0].text

    df["生年月日"] = res2_[0]
    df["調教師"] = res2_[1]
    df["馬主"] = res2_[2]
    df["生産者"] = res2_[3]
    df["産地"] = res2_[4]
    df["セリ取引価格"] = res2_[5]
    df["獲得賞金"] = res2_[6]
    df["通算成績"] = res2_[7]
    df["主な勝鞍"] = res2_[8]
    df["近親馬"] = res2_[9]
    df["馬名"] = uma_name
    
    return df

In [75]:
driver = webdriver.Chrome("chromedriver.exe")
df = uma_get(driver, 'https://db.netkeiba.com/horse/2018105845/')